In [ ]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import PolynomialFeatures

# Regression

In [ ]:
def generateNoisyData(num_points=50):
    x = np.linspace(1, 4*math.pi, num_points)
    y = np.sin(x*0.5)
    nmu, sigma = 0, 0.3
    noise = nmu + sigma * np.random.randn(num_points)
    t = y + noise
    np.savez('data.npz', x=x, y=y, t=t, sigma=sigma)

In [ ]:
def plot_results(x=None, y=None, pred1 = None, label1 = None, sigma = False, pred2 = None, label2 = None, title = None, file_name = None, num_points = 50):
    if not os.path.exists(f'results{num_points}'):
        os.makedirs(f'results{num_points}')
    np.load(f'data{num_points}.npz')
    x = np.load(f'data{num_points}.npz')['x']
    y = np.load(f'data{num_points}.npz')['y']
    t = np.load(f'data{num_points}.npz')['t']
    sigma = np.load(f'data{num_points}.npz')['sigma']
    nmu, sigma = 0, 0.3
    noise = nmu + sigma * np.random.randn(num_points)
    fig, ax = plt.subplots()
    ax.plot(x, y, 'g', label = "Ground Truth")
    ax.plot(x, pred1, 'r', label = label1)
    if pred2 is not None:
        ax.plot(x, pred2, "y", label = label2)
        ax.fill_between(x, y-sigma, y+sigma, color='r', alpha=0.2)
    ax.scatter(x, t, label='Noisy Data Points')
    if pred2 is None:
        ax.fill_between(x, pred1-sigma, pred1+sigma, color='r', alpha=0.2)
        lines = [[(i, j), (i, line)] for i, j, line in zip(x, pred1, t)]    
        lc = mc.LineCollection(lines, colors='red', linewidths=1, zorder=1)
        ax.add_collection(lc)
    ax.set_xlabel('X Values')
    ax.set_ylabel('Y Values, Predictions, Noisy Data')
    ax.set_title(title)
    plt.legend()
    plt.savefig(f'results{num_points}/{file_name}.png')
    plt.show()
    plt.close(fig)

In [ ]:
class ML():
    def __init__(self, degree = None):
        self.degree = degree
    def fit(self, x, y):
        x = PolynomialFeatures(degree = self.degree).fit_transform(x.reshape(-1,1))
        intermediate = (x.T)@x
        weights = np.linalg.solve(intermediate, (x.T)@y)
        return weights
    def predict(self, x, weights):
        x = PolynomialFeatures(degree = self.degree).fit_transform(x.reshape(-1,1))
        predictions = np.dot(x, weights)
        return predictions

In [ ]:
class MAP():
    def __init__(self, alpha=0.005, beta=11.1, lnlambda = None, customReguralization = False, degree = None):
        self.alpha = alpha
        self.beta = beta
        self.lnlambda = lnlambda
        self.customReguralization = customReguralization
        self.degree = degree
    def fit(self, x, y):
        if self.customReguralization:
            lnlambda = self.lnlambda
        else:
            lnlambda = np.log(self.alpha/self.beta)
        x = PolynomialFeatures(degree = self.degree).fit_transform(x.reshape(-1,1))
        intermediate = ((x.T)@x) + np.exp(lnlambda)*np.eye(x.shape[1], x.shape[1])
        weights = np.linalg.solve(intermediate, (x.T)@y)
        return weights
    def predict(self, x, weights):
        x = PolynomialFeatures(degree = self.degree).fit_transform(x.reshape(-1,1))
        predictions = np.dot(x, weights)
        return predictions

In [ ]:
def linear_regression(num_points = 50, calcRMSE = False, rmse_degree = 9):
    np.load(f'data{num_points}.npz')
    x = np.load(f'data{num_points}.npz')['x']
    y = np.load(f'data{num_points}.npz')['y']
    t = np.load(f'data{num_points}.npz')['t']
    sigma = np.load(f'data{num_points}.npz')['sigma']

    ml_weights_dict, map_weights_dict = {}, {}

    for i in [0,1,3,6,9]:
        ML_Model = ML(degree = i)
        ml_weights = ML_Model.fit(x, t)
        ml_predictions = ML_Model.predict(x, ml_weights)
        ml_weights_dict[f"ML:{i}"] = pad_weights(ml_weights)
        plot_results(x, y, ml_predictions, title = f"ML Model Degree {i}", file_name = f"ml_{i}", label1 = "ML Model Prediction", num_points = num_points)
    print(f"Number of Points: {num_points}")
    print(pd.DataFrame(ml_weights_dict))

    for i in [0,1,3,6,9]:
        MAP_Model = MAP(degree = i, customReguralization = False)
        map_weights = MAP_Model.fit(x, t)
        map_predictions = MAP_Model.predict(x, map_weights)
        map_weights_dict[f"MAP:{i}"] = pad_weights(map_weights)
        plot_results(x, y, map_predictions, title = f"MAP Model Degree {i}", file_name = f"map_{i}", label1 = "MAP Model Prediction", num_points = num_points)
    print(f"Number of Points: {num_points}")    
    print(pd.DataFrame(map_weights_dict))

    for i in [3,9,15,20,30,40,50]:
        ML_Model = ML(degree = i)
        MAP_Model = MAP(degree = i)
        ml_weights = ML_Model.fit(x, t)
        map_weights = MAP_Model.fit(x, t)
        ml_predictions = ML_Model.predict(x, ml_weights)
        map_predictions = MAP_Model.predict(x, map_weights)
        plot_results(x, y, pred1 = ml_predictions, label1 = f"ML Model Degree: {i}", \
                        title = f"ML vs MAP Predictions Degree: {i}", \
                        pred2 = map_predictions, label2 = f"MAP Model Degree: {i}", \
                        file_name = f"ml_vs_map{i}", num_points = num_points)

    if calcRMSE:
        rmse_arr = []
        for lnlambda in range(-40, -20):
            inputs = np.load("data50.npz")["x"]
            targets = np.load("data50.npz")["t"]
            rmseModel = MAP(degree = rmse_degree, customReguralization = True, lnlambda = lnlambda)
            weights = rmseModel.fit(inputs, targets)
            predictions = rmseModel.predict(inputs, weights)
            rmse_error = rmse(predictions, targets)
            rmse_arr.append(rmse_error)
        plt.plot(range(-40,-20), rmse_arr)
        # plt.ylim([0, 1])
        plt.xlabel(r"ln$\lambda$")
        plt.ylabel(r"$E_{RMS}$")
        plt.legend()
        plt.show()

    for lnlambda in [-18, -15, -13]:
        CustomModel = MAP(degree = 3, customReguralization = True, lnlambda = lnlambda)
        custom_weights = CustomModel.fit(x, t)
        custom_predictions = CustomModel.predict(x, custom_weights)
        plot_results(x, y, custom_predictions, title = r"Custom Model Degree 3, ln$\lambda$ = " + str(lnlambda), file_name = f"lnlambda{lnlambda}", label1 = r"$ln\lambda$ = "+str(lnlambda)+" Model", num_points = num_points)

In [ ]:
for num_points in [20,50,500]:
    generateNoisyData(num_points = num_points)
    plot_with_shadded_bar(num_points = num_points)
    linear_regression(num_points = num_points, calcRMSE = True)

# Classification

In [ ]:
def viz_desc_bounds(classifier, feats, labels, idxA, idxB):
    if not os.path.exists('results'):
        os.makedirs('results')
    ys = np.sort(np.unique(labels))
    y_ind = np.searchsorted(ys, labels)
    fig, ax = plt.subplots()
    x0, x1 = feats[:, 0], feats[:, 1]
    all_feats = np.concatenate((x0, x1))
    pad = np.percentile(all_feats, 60)
    x_min, x_max = x0.min() - pad, x0.max() + pad
    y_min, y_max = x1.min() - pad, x1.max() + pad
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))
    preds = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
    preds = preds.reshape(xx.shape)
    lut = np.sort(np.unique(labels)) 
    ind = np.searchsorted(lut,preds)
    markers = ["o", "v", "P", "X", "s", "p", "h", "d"]
    ax.contourf(xx, yy, preds, cmap=plt.cm.Pastel1, alpha=0.8)
    for i in range(len(lut)):
        ax.scatter(x0[y_ind == i], x1[y_ind == i], color=plt.cm.jet(i/len(lut)), s=50, edgecolors='k', marker=markers[i])
    ax.set_xlabel(f'Feature {idxA}')
    ax.set_ylabel(f'Feature {idxB}')
    ax.set_title('Decision Boundary')
    handles = []
    markers = ["o", "v", "P", "X", "s", "p", "h", "d"]
    handles = [plt.plot([],[],color=plt.cm.jet(i/len(lut)), ls="", marker=markers[i])[0] for i in range(len(lut))]
    labels = [f'Class {i}' for i in lut]
    ax.legend(handles, labels, loc='upper right')
    plt.savefig(f'results/example_boundary.png')
    plt.show()

In [ ]:
def load_dataset(dataset='taiji', verbose=False, subject_index=3):
    if dataset == 'taiji':
        labels = np.loadtxt("../P1/data/taiji/taiji_labels.csv", delimiter=",", dtype=int)
        person_idxs = np.loadtxt("../P1/data/taiji/taiji_person_idx.csv", delimiter=",", dtype=int)
        feats = np.loadtxt("../P1/data/taiji/taiji_quat.csv", delimiter=",", dtype=float)
        labels[labels == 4] = 2
        labels[labels == 8] = 6
        feature_mask = np.var(feats, axis=1) > 0
        train_mask = person_idxs != subject_index
        train_feats = feats[feature_mask, :][:, train_mask].T
        train_labels = labels[train_mask].astype(int)
        test_feats = feats[feature_mask, :][:, ~train_mask].T
        test_labels = labels[~train_mask].astype(int)
    if verbose:
        print(f'{dataset} Dataset Loaded')
        print(f'\t# of Classes: {len(np.unique(train_labels))}')
        print(f'\t# of Features: {train_feats.shape[1]}')
        print(f'\t# of Training Samples: {train_feats.shape[0]}')
        print('\t# per Class in Train Dataset:')
        for cls in np.unique(train_labels):
            print (f'\t\tClass {cls}: {np.sum(train_labels == cls)}')
        print(f'\t# of Testing Samples: {test_feats.shape[0]}')
        print('\t# per Class in Test Dataset:')
        for clas in np.unique(test_labels):
            print(f'\t\tClass {clas}: {np.sum(test_labels == clas)}')
    return train_feats, train_labels, test_feats, test_labels

In [ ]:
def plot_conf_mats(dataset, **kwargs):
    train_labels = kwargs['train_labels']
    pred_train_labels = kwargs['pred_train_labels']
    test_labels = kwargs['test_labels']
    pred_test_labels = kwargs['pred_test_labels']
    train_confusion = confusion_matrix(train_labels, pred_train_labels)
    test_confusion = confusion_matrix(test_labels, pred_test_labels)
    fig, ax = plt.subplots()
    disp = ConfusionMatrixDisplay(confusion_matrix=train_confusion, display_labels=np.unique(train_labels))
    disp.plot(ax=ax, xticks_rotation='vertical')
    ax.set_title('Training Confusion Matrix')
    plt.tight_layout()
    plt.savefig(f'results/{dataset}_train_confusion.png', bbox_inches='tight', pad_inches=0)
    plt.show()
    plt.close(fig)

    fig, ax = plt.subplots()
    disp = ConfusionMatrixDisplay(confusion_matrix=test_confusion, display_labels=np.unique(test_labels))
    disp.plot(ax=ax, xticks_rotation='vertical')
    ax.set_title('Testing Confusion Matrix')
    plt.tight_layout()
    plt.savefig(f'results/{dataset}_test_confusion.png', bbox_inches='tight', pad_inches=0)
    plt.show()

In [ ]:
def example_decision_boundary(dataset='taiji', indices=[0, 6]):
    train_feats, train_labels, test_feats, test_labels = load_dataset(dataset=dataset)
    dc_train_feats = train_feats[:, indices]
    dc_test_feats = test_feats[:, indices]
    clf = LinearDiscriminantAnalysis()
    clf.fit(dc_train_feats, train_labels)
    viz_desc_bounds(clf, dc_test_feats, test_labels, indices[0], indices[1])

In [ ]:
def example_classification(dataset='taiji'):
    train_feats, train_labels, test_feats, test_labels = load_dataset(dataset=dataset)
    clf = LinearDiscriminantAnalysis()
    clf.fit(train_feats, train_labels)
    pred_train_labels = clf.predict(train_feats)
    pred_test_labels = clf.predict(test_feats)
    plot_conf_mats(dataset, train_labels=train_labels, pred_train_labels=pred_train_labels, test_labels=test_labels, pred_test_labels=pred_test_labels)